# Multi-feature Linear Regression with Tensorflow
### The following code provides the basic skeleton for performing multivariable regression using Tensorflow.
### The code should be optimized for a real dataset! 
The code can be seen as an introduction for the usage of tf.Variable, tf.placeholder, GradientDescentOptimizer, tf.Session etc. for a more complex classification or regression algorithm.

I first implement a simple Regression for a small dataset and later I implement the batch feeding for larger datasets.

In [1]:
import tensorflow as tf
import numpy as np
np.set_printoptions(precision=3)
import matplotlib.pyplot as plt
%matplotlib inline

/Users/param/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Generating the data
The following cell should be modified to input real data

In [2]:
#Setting the data variables

num_features = 4
D = 30 #number of data points = length of the vectors

X_data = np.zeros((D,num_features))
for i in range(num_features):
    X_data[:,i] = np.linspace(0,10,D) + np.random.uniform(-1,1,D)

y_data = np.linspace(0,100,D) + np.random.uniform(-1,1,D)

### Initializing the parameters
With $y = m \cdot x +b$, we have $D+1$ parameters: $m$ a D dimensional vector and $b$ a scalar

In [3]:
m = tf.Variable(initial_value=tf.ones([num_features],tf.float64))
b = tf.Variable(1.0)

### Defining the Loss function 

In [4]:
error = 0
for X,y in zip(X_data,y_data):
    y_hat = tf.tensordot(X,m,1) + tf.cast(b, tf.float64)
    error += (y-y_hat)**2

### Hyperparameters

In [5]:
#Learning rate, should be adjusted for a larger range of features in the dataset
lr = 1e-4 

# relative difference in consecutive losses at which training should stop
stopping = 1e-8

### Tell Tensorflow what to minimize

In [6]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train = optimizer.minimize(error)

### Training

In [7]:
max_steps =10000

#the following command initializes the tf.Variables, for us m and b
init = tf.global_variables_initializer()

#Training
print('Tensorflow implementation:')
with tf.Session() as sess:
    sess.run(init)
  
    print('initial:\t m: {:20}\t b:{:.3f}\t loss:{:.4f}'
          .format(str(sess.run(m)),sess.run(b),sess.run(error)))
    
    loss_last = sess.run(error) + 1.0

    i = 0
    loop = True
    
    while (loop & (i < max_steps)):
        sess.run(train)
        
        if np.isnan(sess.run(error)):
            print('Loss function became NAN, consider decreasing learning rate')
            break
        
        #stopping criteria if the results are not changing
        if ((np.abs(loss_last-sess.run(error))/loss_last < stopping)):
            loop = False 

        i += 1
        loss_last = sess.run(error)
        
        #Print some information
        if ((i+1)%(max_steps/10) ==0):
            print('step:{:4d}\t m: {:20}\t b:{:.3f}\t loss:{:.4f}'
                  .format(i+1,str(sess.run(m)),sess.run(b),sess.run(error)))

    print()
    final_slope = sess.run(m)
    final_intercept = sess.run(b)
    if i < max_steps:
        print('Total no. of steps: {:d}'.format(i))
        print('Final parameters m: {:20}\t b:{:.3f}\t loss:{:.4f}'
              .format(str(sess.run(m)),sess.run(b),sess.run(error)))
    else:
        print('Loss function did not converge according to the set criteria after {:d} steps'.format(max_steps))
        print('Consider increasing max_steps or learning rate variable (lr)')      
        print('Obtained parameters m: {:20}\t b:{:.3f}\t loss:{:.4f}'
              .format(str(sess.run(m)),sess.run(b),sess.run(error)))

Tensorflow implementation:
initial:	 m: [1. 1. 1. 1.]       	 b:1.000	 loss:34145.8193
step:1000	 m: [2.767 2.912 2.793 1.287]	 b:0.458	 loss:226.9882
step:2000	 m: [2.724 3.    2.89  1.182]	 b:0.212	 loss:225.9545
step:3000	 m: [2.705 3.023 2.914 1.164]	 b:0.137	 loss:225.8721

Total no. of steps: 3858
Final parameters m: [2.7   3.029 2.92  1.16 ]	 b:0.117	 loss:225.8654


In [8]:
#Sanity check with Sklearn's function    
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X_data, y_data)
sk_slope = reg.coef_
sk_intercept = reg.intercept_

print('Sklearn implementation: m={:5}  \t b={:.3f} \t loss={:.3f}'
      .format(str(sk_slope),sk_intercept,
              D*mean_squared_error(y_data, reg.predict(X_data))))

Sklearn implementation: m=[2.697 3.032 2.923 1.158]  	 b=0.107 	 loss=225.864


In [9]:
#Visualize the results
if num_features == 1:
    plt.plot(X_data,y_data,'+')
    
    #Tensorflow results
    plt.plot(X_data,final_slope*X_data+final_intercept,'r')
    
    #Sklearn
    plt.plot(X_data,sk_slope*X_data+sk_intercept,'b')
    
    plt.legend(('Data', 'Our Tensorflow implementation', 'from sklearn'),
           loc='upper left')

# Feeding in Batches

In [10]:
#Setting the data variables

num_features = 2
D = 10000 #number of data points = length of the vectors

X_data = np.zeros((D,num_features))
for i in range(num_features):
    X_data[:,i] = np.linspace(0,10,D) + np.random.uniform(-1,1,D)

b0 = np.random.randint(3)
y_data =  b0 + np.linspace(0,10,D) + np.random.uniform(-1,1,D)

In [11]:
m = tf.Variable(initial_value=tf.ones([num_features],tf.float64))
b = tf.Variable(2.0)

In [12]:
batch_size = 10
X_batch = tf.placeholder(tf.float64,shape=(batch_size,num_features),name='Xbatch')
y_batch = tf.placeholder(tf.float64,shape=(batch_size),name='ybatch')

In [13]:
#Learning rate, should be adjusted for a larger range of features in the dataset
lr = 1e-4

#Defining the optimizer
y_hat = tf.tensordot(X_batch,m,1) + tf.cast(b, tf.float64)
error = tf.reduce_sum(tf.square(y_batch-y_hat))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train = optimizer.minimize(error)

In [14]:
epochs = 10000

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(epochs):
        
        rand_ind = np.random.randint(X_data.shape[0],size=batch_size)
        feed = {X_batch:X_data[rand_ind,:],y_batch:y_data[rand_ind]}
        
        _,mstep,bstep,errorstep = sess.run([train,m,b,error],feed_dict=feed)
        
        if np.isnan(errorstep):
            print('Loss function became NAN, consider decreasing learning rate')
            break
            
        if ((i)%(epochs/10) ==0):
            print('epochs:{:4d}\t m: {:20}\t b:{:.3f}\t  loss:{:.4f}'
                  .format(i+1,str(mstep),bstep,errorstep/batch_size))

    print()
    final_slope = mstep
    final_intercept = bstep

    print('Final parameters m: {:20}\t b:{:.3f}\t  loss:{:.4f}'.format(str(mstep),bstep,errorstep/batch_size))


epochs:   1	 m: [0.93  0.927]       	 b:1.989	  loss:38.4515
epochs:1001	 m: [0.499 0.496]       	 b:1.991	  loss:0.2562
epochs:2001	 m: [0.5   0.498]       	 b:2.031	  loss:0.6952
epochs:3001	 m: [0.494 0.497]       	 b:2.058	  loss:0.5215
epochs:4001	 m: [0.496 0.496]       	 b:2.077	  loss:0.3646
epochs:5001	 m: [0.486 0.48 ]       	 b:2.074	  loss:0.5245
epochs:6001	 m: [0.493 0.502]       	 b:2.075	  loss:0.7793
epochs:7001	 m: [0.472 0.491]       	 b:2.074	  loss:0.4562
epochs:8001	 m: [0.473 0.498]       	 b:2.084	  loss:0.4762
epochs:9001	 m: [0.486 0.492]       	 b:2.084	  loss:0.5188

Final parameters m: [0.48  0.489]       	 b:2.085	  loss:0.6732


In [15]:
#Sanity check with Sklearn's function    
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X_data, y_data)
sk_slope = reg.coef_
sk_intercept = reg.intercept_

print('Sklearn implementation: m={:5}  \t b={:.3f} \t loss={:.3f}'
      .format(str(sk_slope),sk_intercept,
              mean_squared_error(y_data, reg.predict(X_data))))

Sklearn implementation: m=[0.491 0.491]  	 b=2.093 	 loss=0.497


In [16]:
#Visualize the results
if num_features == 1:
    rand_ind = np.random.randint(X_data.shape[0],size=100)
    plt.plot(X_data[rand_ind,:],y_data[rand_ind],'+')
    
    #Tensorflow results
    plt.plot(X_data,final_slope*X_data+final_intercept,'r')
    
    #Sklearn
    plt.plot(X_data,sk_slope*X_data+sk_intercept,'b')
    
    plt.legend(('Data', 'Our Tensorflow implementation', 'from sklearn'),
           loc='upper left')